In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import hamming
from scipy.signal import chirp



In [2]:
# Load the datasets
data_0hp = pd.read_csv('0hp_all_faults.csv')
data_1hp = pd.read_csv('1hp_all_faults.csv')
data_2hp = pd.read_csv('2hp_all_faults.csv')




In [3]:
signal_0hp = data_0hp.iloc[:, 0].values
signal_1hp = data_1hp.iloc[:, 0].values
signal_2hp = data_2hp.iloc[:, 0].values

def wvd(signal_segment):
    N = len(signal_segment)
    tfr = np.zeros((N, N), dtype=complex)
    for n in range(N):
        for m in range(-n, N-n):
            tfr[n, (n+m) % N] += signal_segment[n] * np.conj(signal_segment[(n-m) % N])
    return tfr

def pwvd(signal_segment):
    N = len(signal_segment)
    window = hamming(N)
    tfr = np.zeros((N, N), dtype=complex)
    for n in range(N):
        for m in range(-n, N-n):
            tfr[n, (n+m) % N] += window[m] * signal_segment[n] * np.conj(signal_segment[(n-m) % N])
    return tfr

def spwvd(signal_segment):
    N = len(signal_segment)
    window = hamming(N)
    smoothed_tfr = np.zeros((N, N), dtype=complex)
    for n in range(N):
        for m in range(-n, N-n):
            smoothed_tfr[n, (n+m) % N] += window[m] * window[n] * signal_segment[n] * np.conj(signal_segment[(n-m) % N])
    return smoothed_tfr

def plot_tfr(tfr, title):
    plt.figure(figsize=(10, 6))
    plt.imshow(np.abs(tfr), aspect='auto', origin='lower', cmap='jet')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.colorbar()
    plt.show()



In [4]:
def process_signal(signal, segment_size=1024):
    num_segments = len(signal) // segment_size
    aggregated_tfr = None
    
    for i in range(num_segments):
        segment = signal[i * segment_size: (i + 1) * segment_size]
        tfr = spwvd(segment)  # Use spwvd, change to wvd or pwvd if needed
        if aggregated_tfr is None:
            aggregated_tfr = tfr
        else:
            aggregated_tfr = np.hstack((aggregated_tfr, tfr))
    
    return aggregated_tfr



In [ ]:
# Process and plot time-frequency representations for 0hp signal
tfr_result_0hp = process_signal(signal_0hp, segment_size=1024)
plot_tfr(tfr_result_0hp, 'Smoothed Pseudo-Wigner-Ville Distribution (0hp)')

# Process and plot time-frequency representations for 1hp signal
tfr_result_1hp = process_signal(signal_1hp, segment_size=1024)
plot_tfr(tfr_result_1hp, 'Smoothed Pseudo-Wigner-Ville Distribution (1hp)')

# Process and plot time-frequency representations for 2hp signal
tfr_result_2hp = process_signal(signal_2hp, segment_size=1024)
plot_tfr(tfr_result_2hp, 'Smoothed Pseudo-Wigner-Ville Distribution (2hp)')